In [1]:
# If you run this notebook on Google Colaboratory, uncomment the below to install automl_alex.
#!pip install --quiet -U automl_alex

In [2]:
import automl_alex
import sklearn
import time
from automl_alex import AutoML, AutoMLClassifier
print(automl_alex.__version__)

0.07.24


In [3]:
RANDOM_SEED = 42

# Classifier

## Data

In [4]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
dataset = fetch_openml(name='credit-g', version=1, as_frame=True)
dataset.target = dataset.target.astype('category').cat.codes
dataset.data.head(5)

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
0,<0,6.0,critical/other existing credit,radio/tv,1169.0,no known savings,>=7,4.0,male single,none,4.0,real estate,67.0,none,own,2.0,skilled,1.0,yes,yes
1,0<=X<200,48.0,existing paid,radio/tv,5951.0,<100,1<=X<4,2.0,female div/dep/mar,none,2.0,real estate,22.0,none,own,1.0,skilled,1.0,none,yes
2,no checking,12.0,critical/other existing credit,education,2096.0,<100,4<=X<7,2.0,male single,none,3.0,real estate,49.0,none,own,1.0,unskilled resident,2.0,none,yes
3,<0,42.0,existing paid,furniture/equipment,7882.0,<100,4<=X<7,2.0,male single,guarantor,4.0,life insurance,45.0,none,for free,1.0,skilled,2.0,none,yes
4,<0,24.0,delayed previously,new car,4870.0,<100,1<=X<4,3.0,male single,none,4.0,no known property,53.0,none,for free,2.0,skilled,2.0,none,yes


In [5]:
X_train, X_test, y_train, y_test = train_test_split(dataset.data, 
                                                    dataset.target,
                                                    test_size=0.25, 
                                                    random_state=RANDOM_SEED,)
X_train.shape, X_test.shape

((750, 20), (250, 20))

## Model

In [6]:
model = AutoML(X_train, y_train, X_test, type_of_estimator='classifier', random_state=RANDOM_SEED)

# or Simply
model = AutoMLClassifier(X_train, y_train, X_test, random_state=RANDOM_SEED, verbose=1)

Source X_train shape:  (750, 20) | X_test shape:  (250, 20)
##################################################
Auto detect cat features:  13
> Start preprocessing Data
> Generate cat encodet features
 +  55  Features from  OneHotEncoder
 +  44  Features from  HelmertEncoder
 +  54  Features from  HashingEncoder
 +  16  Features from  FrequencyEncoder
> Generate Frequency Encode num features
 +  4  Frequency Encode Num Features 
> Clean Nans in num features
> Generate interaction Num Features
 +  24  Interaction Features
> Normalization Features
##################################################
> Total Features:  201
##################################################
New X_train shape:  (750, 201) | X_test shape:  (250, 201)


In [7]:
%%time
predict_test, predict_train = model.fit_predict(timeout=1000, verbose=2)


 Opt BestModels
One iteration takes ~ 0.4 sec
> Start Auto calibration parameters
> Start optimization with the parameters:
CV_Folds =  10
Score_CV_Folds =  5
Feature_Selection =  True
Opt_lvl =  3
Cold_start =  83.0
Early_stoping =  166.0
Metric =  roc_auc_score
Direction =  maximize
##################################################
Default model OptScore = 0.724
Optimize: : 178it [09:22,  3.16s/it,  | Model: CatBoost | OptScore: 0.784 | Best roc_auc_score: 0.825 +- 0.040991]

 Predict from Models_1
 12%|█▎        | 1/8 [00:13<01:32, 13.18s/it]
 Mean Score roc_auc_score on 20 Folds: 0.7962 std: 0.048919
 25%|██▌       | 2/8 [00:19<01:06, 11.10s/it]
 Mean Score roc_auc_score on 20 Folds: 0.7997 std: 0.047945
 38%|███▊      | 3/8 [01:02<01:43, 20.63s/it]
 Mean Score roc_auc_score on 20 Folds: 0.7935 std: 0.044447
 50%|█████     | 4/8 [01:10<01:07, 16.76s/it]
 Mean Score roc_auc_score on 20 Folds: 0.8015 std: 0.05078
 62%|██████▎   | 5/8 [01:16<00:40, 13.61s/it]
 Mean Score roc_auc_sco

In [8]:
predict_test[:5]

array([0.3688743 , 0.45752244, 0.66583246, 0.28677225, 0.09116841])

In [9]:
print('Test AUC: ', round(sklearn.metrics.roc_auc_score(y_test, predict_test),4))

Test AUC:  0.8133


In [10]:
model.stack_models_cfgs

,score_opt,model_score,score_std,model_name,model_param,wrapper_params,cat_encoders,columns,cv_folds
0,0.7840,0.8250,0.040991,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[num_dependents, OneHotEncoder_savings_status_...",10
1,0.7798,0.8232,0.043370,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[duration, num_dependents, OneHotEncoder_savin...",10
2,0.7780,0.8112,0.033221,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[duration, age, num_dependents, OneHotEncoder_...",10
3,0.7778,0.8214,0.043639,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[duration, num_dependents, OneHotEncoder_savin...",10
4,0.7770,0.8221,0.045123,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[duration, num_dependents, OneHotEncoder_savin...",10
5,0.7762,0.8083,0.032123,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[OneHotEncoder_savings_status_2, OneHotEncoder...",10
6,0.7757,0.8184,0.042703,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[duration, num_dependents, OneHotEncoder_savin...",10
7,0.7751,0.8105,0.035414,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[duration, age, OneHotEncoder_savings_status_2...",10
8,0.7354,0.7896,0.054184,LinearModel,"{'verbose': 0, 'fit_intercept': True, 'C': 52....",{},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[duration, age, OneHotEncoder_savings_status_3...",10
9,0.7333,0.7874,0.054064,LinearModel,"{'verbose': 0, 'fit_intercept': True, 'C': 66....",{},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[age, OneHotEncoder_savings_status_1, OneHotEn...",10
